In [2]:
from selenium import webdriver
import pandas as pd

In [86]:
driver = webdriver.Chrome('/Users/prajvalgupta/Downloads/chromedriver 2')

In [111]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

driver = webdriver.Chrome('/Users/prajvalgupta/Downloads/chromedriver 2')

beer_dict={}
driver.get('https://www.beeradvocate.com/beer/top-rated/')
beer_profiles=[]
beer_profile=driver.find_element_by_xpath('//div[@id="ba-content"]/table/tbody/tr[2]/td[2]/a')
# print(beer_profile.get_attribute('href'))
for i in range(2,252):
    beer=driver.find_element_by_xpath('//div[@id="ba-content"]/table/tbody/tr['+str(i)+']/td[2]/a')
    beer_name=driver.find_element_by_xpath('//div[@id="ba-content"]/table/tbody/tr['+str(i)+']/td[2]/a/b').text
    beer_dict[beer_name]=beer.get_attribute('href')

driver.close()

comments = pd.DataFrame(columns = ['Product Name','Product Rating','Product Review']) 

try:
    for key,value in beer_dict.items():
        driver = webdriver.Chrome('/Users/prajvalgupta/Downloads/chromedriver 2')
        driver.get(value)
        reviews = driver.find_elements_by_xpath('//div[@id="rating_fullview_content_2"]')
        for review in reviews:
            review_text = review.text

            rating = review.find_element_by_xpath('.//span[1]').text
            extra1 = review.find_element_by_xpath('.//span[2]').text
            extra2 = review.find_element_by_xpath('.//span[3]').text
            extra3 = review.find_element_by_xpath('.//span[4]').text
            extra4 = review.find_element_by_xpath('.//span[5]').text
            extra5 = review.find_element_by_xpath('.//div').text

            if extra3:
                review_text = review_text.replace(extra3,"")
            if extra1:
                review_text = review_text.replace(extra1,"")
            if extra2:
                review_text = review_text.replace(extra2,"")
            if extra4:
                review_text = review_text.replace(extra4,"")
            if extra5:
                review_text = review_text.replace(extra5,"")

    #         print(review_text)

            comments.loc[len(comments)] = [key,rating,review_text]

        driver.close()
except:
    pass

In [113]:
comments.to_csv("product_reviews.csv")

In [27]:
from nltk import ngrams, FreqDist
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, webtext
from sklearn import manifold
import nltk.data
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sentiment

In [3]:
df=pd.read_csv('product_reviews.csv')

In [4]:
df.head()


,Unnamed: 0,Product Name,Product Rating,Product Review
0,0,Kentucky Brunch Brand Stout,4.80,4.8 rDev \n\n\n2016 Silver Wax. Aroma has whi...
1,1,Kentucky Brunch Brand Stout,4.74,4.74 rDev \n\n\nThe beer pours Pitch Black wi...
2,2,Kentucky Brunch Brand Stout,4.68,4.68 rDev \n\n\nProbably the smoothest beer I...
3,3,Kentucky Brunch Brand Stout,5.00,"5 rDev \n\n\nDark black, very thick, a little..."
4,4,Kentucky Brunch Brand Stout,4.97,4.97 rDev \n\n\nPoured black as ink with thin...


In [11]:
df_sim=df.iloc[0:300]
df_sim.head()

,Unnamed: 0,Product Name,Product Rating,Product Review
0,0,Kentucky Brunch Brand Stout,4.80,4.8 rDev \n\n\n2016 Silver Wax. Aroma has whi...
1,1,Kentucky Brunch Brand Stout,4.74,4.74 rDev \n\n\nThe beer pours Pitch Black wi...
2,2,Kentucky Brunch Brand Stout,4.68,4.68 rDev \n\n\nProbably the smoothest beer I...
3,3,Kentucky Brunch Brand Stout,5.00,"5 rDev \n\n\nDark black, very thick, a little..."
4,4,Kentucky Brunch Brand Stout,4.97,4.97 rDev \n\n\nPoured black as ink with thin...


In [12]:
df_sim.drop('Unnamed: 0', axis=1, inplace=True)

In [23]:
df_sim['CleanReview']=df_sim['Product Review'].apply(lambda x: x.split('\n')[3])

/Users/subhayuchakravarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
df_sim['Product Name'].unique()

array(['Kentucky Brunch Brand Stout', 'Marshmallow Handjee',
       "Hunahpu's Imperial Stout - Double Barrel Aged",
       'Barrel-Aged Abraxas', 'Heady Topper', 'King Julius',
       'Pliny The Younger', "Mornin' Delight", 'Very Hazy',
       'King JJJuliusss', 'Fundamental Observation', 'SR-71'],
      dtype=object)

In [28]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/subhayuchakravarty/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
df_sim[]

In [32]:
sorted_lifts=[(('Kentucky Brunch Brand Stout','smooth'),1.2),(('Kentucky Brunch Brand Stout','aggresive'),1.1),(('Pliny The Younger','smooth'),1.8),(('Pliny The Younger','aggresive'),0.8),(('King JJJuliusss','aggresive'),3.8)]

In [34]:
def sentiment_value(review):
    analyser = sentiment()
    result = analyser.polarity_scores(review)
    score = result['compound']
    return round(score,1)
count=0
for i in sorted_lifts:
    candidate = i[0][0]
    place = i[0][1]
    key = (candidate,place)
    sentiments = []
    for review in df_sim['CleanReview']:
        if candidate in review and place in review:
            words = re.findall(r'\w+', review)
            if place in words:
                index = words.index(place)
                left = words[index - 3:index]
                right = words[index + 1:index + 4]
                tweetyy = " ".join(left) + " " + words[index] + " " + " ".join(right)
            else:
                tweetyy = tweet
            sv = sentiment_value(tweetyy)
            sentiments.append(sv)
    print(sentiments)
    print(candidate)
    avg_sentiment = round(sum(sentiments)/len(sentiments),2)
    sentiment_dict[key] = avg_sentiment

print("Beer Sentiment")
sentiment_dict

[]
Kentucky Brunch Brand Stout


ZeroDivisionError: division by zero